## Train GPT Model on JSON data

In [ ]:
# include ../ in path
import sys
sys.path.insert(0, '../')

In [ ]:
# Imports and Setup
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
from tqdm.notebook import tqdm
import psutil
import gc
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Check if MPS is available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
from docformer.data import load_from_mongodb

train_dataset, test_dataset = load_from_mongodb("mongodb://localhost:27017/", 
                                                db_name="openml", coll_name="1590-adult", 
                                                target_field="target", test_size=0.2, verbose=True)

In [ ]:
from docformer.data import tokenize

# Load and prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# tokenize datasets
train_tokenized = tokenize(tokenizer, train_dataset)
test_tokenized = tokenize(tokenizer, test_dataset)


# Create custom config
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=128,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    hidden_size=128, 
    num_attention_heads=4,
    num_hidden_layers=4,
)

model = GPT2LMHeadModel(config).to(device)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
from transformers import TrainerCallback

class LossCallback(TrainerCallback):
    def __init__(self):
        self.training_loss = []
        self.step = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and 'loss' in logs:
            self.training_loss.append(logs['loss'])
            self.step.append(state.global_step)
            self.plot_loss()

    def plot_loss(self):
        clear_output(wait=True)
        plt.figure(figsize=(12, 6))
        plt.plot(self.step, self.training_loss)
        plt.title('Training Loss Curve')
        plt.xlabel('Step')
        plt.ylabel('Loss')
        plt.show()

    # Implement other required methods as no-ops
    def on_init_end(self, args, state, control, **kwargs):
        pass

    def on_train_begin(self, args, state, control, **kwargs):
        pass

    def on_train_end(self, args, state, control, **kwargs):
        pass

loss_callback = LossCallback()

In [ ]:

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none",  # Disable wandb logging
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    callbacks=[loss_callback]
)


In [ ]:

# Train model
print("Starting training...")
trainer.train()
print("Training completed!")
loss_callback.plot_loss()


In [ ]:

# Save the model
model.save_pretrained("./checkpoints/checkpoint-final")
tokenizer.save_pretrained("./checkpoints/checkpoint-final")
print("Model saved!")


In [ ]:

# Memory cleanup
del model, trainer, tokenizer, train_tokenized, test_tokenized, dataset, train_dataset, test_dataset, dataset_split
gc.collect()
print(f"Current memory usage: {psutil.virtual_memory().percent}%")

## Evaluation

In [ ]:
model = GPT2LMHeadModel.from_pretrained("./checkpoints/checkpoint-final")
tokenizer = AutoTokenizer.from_pretrained("./checkpoints/checkpoint-final")

In [ ]:
from transformers import Trainer, TrainingArguments
import numpy as np
import torch

# Set up evaluation arguments
eval_args = TrainingArguments(
    output_dir="./eval_results",
    do_train=False,
    do_eval=True,
    per_device_eval_batch_size=32,
    dataloader_drop_last=False
)

# Create a Trainer instance for evaluation
eval_trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=test_tokenized,
    data_collator=data_collator
)

# Run evaluation
eval_results = eval_trainer.evaluate()

# Print evaluation results
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

# Calculate perplexity
perplexity = np.exp(eval_results['eval_loss'])
print(f"Perplexity: {perplexity}")

# Generate some text as a sanity check
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
_ = model.eval()


In [ ]:
from docformer.eval_batch import evaluate_gpt2_classification

model_path = "./checkpoints/checkpoint-final"
results = evaluate_gpt2_classification(model_path, test_dataset)
print(f"Classification Accuracy: {results['accuracy']:.2f}")